## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`. 
Задание выполнить несколькими способами ("naive" Python, numpy, с использованием numba декоратора @njit) Проверить время выполнения каждого способа %time или %%time

2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв. Задание выполнить несколькими способами ("naive" Python, numpy, с использованием numba декоратора @njit - оценить возможность). Проверить время выполнения каждого способа %time или %%time

## Лабораторная работа 3

In [1]:
import pandas as pd
import numpy as np

###### 1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [3]:
recipes = pd.read_csv('data/recipes_sample.csv')
reviews = pd.read_csv('data/reviews_sample.csv', index_col=0)

In [4]:
reviews.date = pd.to_datetime(reviews.date)
my_reviews = reviews[reviews.date.dt.year == 2010]

### a

In [31]:
%%timeit


ex = list()

for index, row in reviews.iterrows():
    if row.date.year == 2010:
        ex.append(row.rating)
        
sum(ex) / len(ex)     

3.4 s ± 42.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [38]:
%%prun


ex = list()

for index, row in reviews.iterrows():
    if row.date.year == 2010:
        ex.append(row.rating)
        
sum(ex) / len(ex)     

### b

In [33]:
%%timeit

ex = list()
for index, row in my_reviews.iterrows():
    ex.append(row.rating)
    
sum(ex) / len(ex)

905 ms ± 146 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### v

In [37]:
%%timeit

reviews.rating.mean()

258 µs ± 8.36 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [44]:
%lprun?

In [46]:
from a_solution import a_solution

In [47]:
%lprun -f a_solution a_solution(reviews)

### a

In [41]:
%%writefile a_solution.py

def a_solution(reviews):
    ex = list()

    for index, row in reviews.iterrows():
        if row.date.year == 2010:
            ex.append(row.rating)

    return sum(ex) / len(ex)     

Writing a_solution.py


Большую часть времени (80%) тратится на итерирование по строкам таблицы и 17% времени на сравнения года

### b

In [42]:
%%writefile b_solution.py


def b_solution(my_reviews):
    ex = list()
    for index, row in my_reviews.iterrows():
        ex.append(row.rating)

    return sum(ex) / len(ex)

Writing b_solution.py


In [49]:
from b_solution import b_solution

In [50]:
%lprun -f b_solution b_solution(my_reviews)

Аналогично с пунктом а, большая часть времени тратится на итерирование по строкам.

### fixed b

In [57]:
%%writefile fixed_b_solution.py

def fixed_b_solution(my_reviews):
    ex = list()
    for el in my_reviews.rating:
        ex.append(el)
        
    return sum(ex) / len(ex)

Writing fixed_b_solution.py


In [58]:
from fixed_b_solution import fixed_b_solution 

In [61]:
%%timeit 

fixed_b_solution(my_reviews)

1.12 ms ± 443 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [62]:
%lprun -f fixed_b_solution fixed_b_solution(my_reviews)

### v

In [43]:
%%writefile v_solution.py

def v_solution(reviews):
    return reviews.rating.mean()

Writing v_solution.py


In [51]:
from v_solution import v_solution

In [53]:
%lprun -f v_solution v_solution(reviews)

###### 3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [68]:
%%writefile word_count.py


def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

Writing word_count.py


In [66]:
%%timeit

get_word_reviews_count(my_reviews)

1.31 s ± 684 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [69]:
%lprun -f get_word_reviews_count get_word_reviews_count(my_reviews)

Во-первых, нет смысла два раза итерироваться по одному и тому же обьекту

Во-вторых, можно заменить iterrows на итерирование по Series

### fixed_solution

In [ ]:

def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

In [10]:
%%writefile get_word_reviews.py


def get_word_reviews_count(df):
    word_reviews_count = dict()
    for description in df.review:
        words = description.split(' ')
        for word in words:
            if word not in word_reviews_count:
                word_reviews_count[word] = 0
            word_reviews_count[word] += 1
            
    return word_reviews_count

Writing get_word_reviews.py


In [11]:
from get_word_reviews import get_word_reviews_count

In [12]:
%%timeit

get_word_reviews_count(my_reviews)

117 ms ± 1.91 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
%lprun -f get_word_reviews_count get_word_reviews_count(my_reviews)

In [14]:
%load_ext line_profiler

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [449]:
given_id = 27208

In [450]:
array = np.array(reviews[(reviews.recipe_id == given_id) & (reviews.rating != 0)].rating)

value = reviews[reviews.recipe_id == given_id].rating.iloc[0]

# A

In [454]:
def a_mape(rarr, mean):
    n = 0
    summa = 0
    for el in rarr:
        summa += np.abs((el - mean) / el)
        n += 1
    
    return 100 * (summa / n)
        
    

In [455]:
%%timeit

a_mape(array, value)

2.72 ms ± 198 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# B

In [456]:
from numba import njit, jit, vectorize

In [457]:
@njit
def b_mape(rarr, mean):
    n = 0
    summa = 0
    for el in rarr:
        summa += np.abs((el - mean) / el)
        n += 1
    
    return 100 * (summa / n)    

In [459]:
%%timeit

b_mape(array, value)

4.72 µs ± 64.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


# C

In [460]:
def b_mape(rarr, mean):
    n = 0
    summa = 0
    for el in rarr:
        summa += np.abs((el - mean) / el)
        n += 1
    
    return 100 * (summa / n) 

In [461]:
c_mape = np.vectorize(b_mape, signature='(i),()->()')

In [462]:
%%timeit

c_mape(array, value)

2.81 ms ± 71.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# D

In [483]:
import numba
from numba import vectorize, guvectorize, float64, int64

In [484]:
numba.typeof(array[0])

int64

In [488]:

@guvectorize(['void(int64[:], int64, float64[:])'], '(i),()->()')
def d_mape(rarr, mean, out):
    n = 0
    summa = 0
    for el in rarr:
        summa += np.abs((el - mean) / el)
        n += 1
    
    out[0] = 100 * (summa / n) 

In [489]:
out =  np.ndarray(1)

In [490]:
%%timeit

d_mape(array, value, out)

7.91 µs ± 107 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [439]:
out

array([44.44444444])